In [1]:
from keras.models import load_model
from collections import deque
import numpy as np
import pickle
import cv2

Using TensorFlow backend.
E:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type

In [2]:
model = load_model(r"D:\VSCODE\Git Upload\Sports-Video-Classification\model\videoClassificationModel")
lb = pickle.loads(open(r"D:\VSCODE\Git Upload\Sports-Video-Classification\model\videoclassificationbinarizer.pickle", "rb").read())

mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
Queue = deque(maxlen=128)

In [3]:
outputvideo = r"D:\VSCODE\Git Upload\Sports-Video-Classification\video\output_video.avi"

In [4]:
capture_video = cv2.VideoCapture(r"D:\VSCODE\Git Upload\Sports-Video-Classification\video\input_video.mp4")
writer = None
(Width, Height) = (None, None)

while True:
    (taken, frame) = capture_video.read() 
    if not taken:
        break
    if Width is None or Height is None:
        (Width, Height) = frame.shape[:2]
        
    output = frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 224)).astype("float32")
    frame -= mean
    
    preds = model.predict(np.expand_dims(frame, axis=0))[0]
    Queue.append(preds)
    
    results = np.array(Queue).mean(axis = 0)
    i = np.argmax(results)
    label  = lb.classes_[i]
    
    text = "They are Playing : {}".format(label)
    cv2.putText(output, text, (45, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (255, 0, 0), 5)
    
    if writer is None:
    #    fourcc = cv2.VideoWriter_fourcc("MJPG")
        writer = cv2.VideoWriter("outputvideo", fourcc, 30, (Width, Height), True)
    writer.write(output)
    cv2.imshow("In progress", output)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord('q'):
        break
        
writer.release()
capture_video.release()